In [1]:
import os
import sys

import json

import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt

from lightning.pytorch.callbacks import ModelCheckpoint

import torch

from dotenv import load_dotenv

from lightning.pytorch.loggers import WandbLogger
import wandb

import session_info
import warnings
from pyprojroot.here import here

#plt.style.use(['science','nature','no-latex'])
dpi_fig_save = 300
sc.set_figure_params(dpi=100, dpi_save=dpi_fig_save, vector_friendly=True)

# Setting some parameters
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedKFold

overwriteData = True
overwriteFigures = True

# Set random seed
random_seed = 42

import warnings
warnings.filterwarnings('ignore')

import scvi
scvi.settings.dl_num_workers = 0
scvi.settings.seed = random_seed

#torch.set_float32_matmul_precision('high')
#torch.set_printoptions(precision=3, sci_mode=False, edgeitems=7)
# torch.multiprocessing.set_sharing_strategy('file_system')

%load_ext autoreload
%autoreload 2

/home/jupyter/conda_envs/scarches/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 captum (see https://github.com/pytorch/captum).
INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


In [ ]:
assert load_dotenv()

In [ ]:
workingDir = here('03_downstream_analysis/04_integration_with_annotation/')
workingDir

In [ ]:
class CustomWandbLogger(WandbLogger):
    @property
    def save_dir(self):
        """Gets the save directory.

        Returns:
            The path to the save directory.

        """
        return self.experiment.dir

In [ ]:
scvi.__version__

# Loading data


In [ ]:
# Load the h5ad file
adata = sc.read_h5ad(here("03_downstream_analysis/04_integration_with_annotation/04_MAIN_geneUniverse_noRBCnPlatelets.h5ad"))#, 
                     #backed='r+', chunk_size=50000)


In [ ]:
adata.obs['binned_age'] = adata.obs['binned_age'].astype(str)

#### Preparing scANVI training

In [ ]:
scvi_model = sca.models.SCVI.load(here(f"{workingDir}/results/scVI_model_pretreined_noRBCnPlat/"), adata=adata) 

### Fine tuning with scANVI

**Parameters**

In [ ]:
scANVI_trainer_kwargs = dict(
    n_samples_per_label = None,
    check_val_every_n_epoch = None,
    train_size = 0.8,
    validation_size = 0.2,
    shuffle_set_split = True,
    checkpointing_monitor = 'elbo_validation',
    early_stopping_monitor = 'reconstruction_loss_validation',
    early_stopping_patience = 2,
    early_stopping_min_delta=0.1,
    early_stopping = True,
    max_epochs = 1000,
)
# https://docs.scvi-tools.org/en/stable/api/reference/scvi.train.TrainingPlan.html#scvi.train.TrainingPlan
plan_kwargs = dict(
    lr = 5e-5,
    #reduce_lr_on_plateau = True
)
datasplitter_kwargs = dict(pin_memory=False)
scanvi_parameter_dict = scANVI_trainer_kwargs | plan_kwargs | datasplitter_kwargs

In [ ]:
run_name = f"MAINobj_scANVI_fineTuning_lowLR_noRBCnPlat"
run_name

In [ ]:
scanvi_model = sca.models.SCANVI.from_scvi_model(scvi_model, unlabeled_category = "unknown")
scanvi_model

In [ ]:
logger = CustomWandbLogger(name = run_name, project='inflammation_atlas_R1_scANVI', config = scanvi_parameter_dict)

In [ ]:
model_checkpoint = ModelCheckpoint(monitor='elbo_validation', every_n_epochs=1, save_last=True, save_top_k = -1, 
                                   dirpath=f"{workingDir}/results/scANVI_model_fineTuned_lowLR_noRBCnPlat_checkpoints/")

In [ ]:
try:
    scanvi_model.train(logger=logger, plan_kwargs = plan_kwargs, datasplitter_kwargs=datasplitter_kwargs, 
                       enable_checkpointing=True, callbacks=[model_checkpoint], 
                       **scANVI_trainer_kwargs) #SAME parameter as scVI
    scanvi_model.save(here(f"{workingDir}/results/scANVI_model_fineTuned_lowLR_noRBCnPlat/"),
               overwrite = True, 
               save_anndata = False)
    scanvi_emb = scanvi_model.get_latent_representation(adata=adata)
    np.savez_compressed(file = str(here(f"{workingDir}/results/scANVI_model_fineTuned_lowLR_noRBCnPlat/scANVI_embedding.npz")), arr=scanvi_emb)
    
except Exception as e:
    print(f"An error occurred: {e}")
    scanvi_model.save(here(f"{workingDir}/results/scANVI_model_fineTuned_lowLR_noRBCnPlat_WITHERRORS/"), 
                      overwrite=True, 
                      save_anndata=False)

In [ ]:
wandb.finish()